# Rotator Streanous

This notebook was prepared to do the testing of the rotator in coordination with the pointing kernel and the camera cable wrap.
The goal is to send command to the rotator to go to 90 at azimuth = 0, wait for a minute and then send the rotator to -90 at azimuth = 180

In [2]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime

In [3]:
now = datetime.now()
test_message = "Rotator integration test, Streneous motion"
print(test_message, now)

Rotator integration test, Streneous motion 2019-12-12 19:00:29.114127


In [4]:
d = salobj.Domain()

In [5]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [6]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [7]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [8]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 15 messages


[None, None, None, None]

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

# Start at 0

In [10]:
now = datetime.now()
print("Start at 0", now)

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 30s")
await asyncio.sleep(30.)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages


Start at 0 2019-12-12 19:04:11.399453
Time error=-57.15 sec


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages


Alt: 45.0 deg
Az: 0.0 deg
RotTel: 0.0 deg
ParaAng: 179.73202218648962 deg
0d00m00s


RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 51 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


Waiting 30s


In [11]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

await asyncio.sleep(0.5)

# Start Test

In [12]:
now = datetime.now()
print("Start Test", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

Start Test 2019-12-12 19:05:07.590579


True

# Go to 90 and then -90

In [13]:
now = datetime.now()
print("Azimuth = 0 and rot_tel= 89 deg", now)

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 35s")
await asyncio.sleep(35.)

#Workaround for Rotator failing between targets
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)
await asyncio.sleep(0.5)
    
    
# # ########################################################################################### 
now = datetime.now()    
print("Azimuth = 180 and rot_tel= -89 deg", now)

alt = 45. * u.deg
az = 180. * u.deg
rot_tel = Angle(-89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

#Workaround for Rotator failing between targets
await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 65s")
await asyncio.sleep(65.)

#Workaround for Rotator failing between targets
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)
await asyncio.sleep(0.5)
    
# ########################################################################################### 
       

RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 96 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages


Azimuth = 0 and rot_tel= 89 deg 2019-12-12 19:05:07.626580


RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 97 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 99 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 99 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 20 messages


Time error=-56.52 sec
Alt: 45.0 deg
Az: 0.0 deg
RotTel: 89.0 deg
ParaAng: 179.73211091595354 deg
88d43m55.5993s
Waiting 35s
Azimuth = 180 and rot_tel= -89 deg 2019-12-12 19:05:50.129071
Time error=-37.00 sec
Alt: 45.0 deg
Az: 180.0 deg
RotTel: -89.0 deg
ParaAng: 0.7348840502184718 deg
-268d15m54.4174s
Waiting 65s


AckTimeoutError: msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=2020867033, ack=<SalRetCode.CMD_ACK: 300>, error=0, result='SAL ACK')

# Back to park position

In [14]:
now = datetime.now()
print("Move to park position", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

#Workaround for Rotator failing between targets
await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)



RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountInPosition) falling behind; read 20 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages


Move to park position 2019-12-12 19:07:30.711617


RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 26 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages


Time error=-56.95 sec
0d00m00s


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1985962330, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Low-level controller in substate 2.0 instead of <EnabledSubstate.STATIONARY: 0>')

In [ ]:
now = datetime.now()
print("Stop tracking",now)
#     #we need to stop the pointing commands before stopping the rotator. This will be integrated in the pointing code
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Test Complete

In [ ]:
now = datetime.now()
print("Test Complete",now)

script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"END - {test_message}")

# Additional Error Handling Commands

# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

In [9]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 33 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 33 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 16 messages


[<State.OFFLINE: 4>,
 <State.STANDBY: 5>,
 <State.DISABLED: 1>,
 <State.ENABLED: 2>]

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)